In [ ]:
from inference.models.faster_rcnn_module import FasterRCNNModule
import numpy as np

from pathlib import Path
import torch
from omegaconf import OmegaConf
import progressbar as pb

model_path = Path('../lightning_logs/art_detection/frcnn_resnet50_img_576.pt')
dataset_path = Path('/home/szymswiat/datasets/nih_dataset')

state = torch.load(model_path.as_posix())
hparams = OmegaConf.create(state['hparams'])
classes = OmegaConf.to_object(hparams.dynamic.classes)

model = FasterRCNNModule.create_from_state(state)
model.model.roi_heads.score_thresh = 0.5

model.eval();

In [ ]:
from data import nih_const
from albumentations.pytorch import ToTensorV2
from data.nih_dataset import NIHDataset
import albumentations as A

metadata = NIHDataset.parse_dataset_meta(
    dataset_path=dataset_path,
    split_type=NIHDataset.SPLIT_OFFICIAL_VAL_FROM_TEST,
    classes=classes
)
img_size = hparams.phases[0].image_size
transforms = A.Compose([
    A.Resize(img_size, img_size),
    A.Normalize(mean=[0] * 3, std=[1] * 3, max_pixel_value=nih_const.MIN_MAX_VALUE[1], always_apply=True),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.2))

test_set = NIHDataset(
    dataset_path=dataset_path,
    input_df=metadata['test_df'],
    mode=NIHDataset.BBOX_ART_DETECTION_MODE,
    transforms=transforms
)

In [ ]:
from torchvision.utils import draw_bounding_boxes
from imageio.plugins.pillow import ndarray_to_pil

for i, (img, y_true) in pb.progressbar(enumerate(test_set)):
    if len(y_true) > 0:
        true_boxes = torch.tensor(y_true)[..., :4]
        true_labels = torch.tensor(y_true)[..., 4]
    else:
        true_boxes = torch.tensor([])
        true_labels = torch.tensor([])
    img_raw = (img * 255).to(torch.uint8)

    y_pred = model(torch.unsqueeze(img, dim=0))
    pred_boxes = y_pred[0]['boxes']
    pred_labels = y_pred[0]['labels']
    pred_scores = y_pred[0]['scores']

    img_raw = draw_bounding_boxes(img_raw, true_boxes,
                              labels=[classes[int(x)] for x in true_labels],
                              colors=[(255, 0, 0)] * len(true_boxes))
    img_raw = draw_bounding_boxes(img_raw, pred_boxes,
                              labels=[f'{classes[int(l)]}: {float(p)}' for l, p in zip(pred_labels, pred_scores)],
                              colors=[(0, 180, 0)] * len(pred_boxes))
    ndarray_to_pil(img_raw.permute(1, 2, 0).detach().numpy()).save(f'images/{test_set.get_img_path(i).stem}.png')

In [ ]:
import cv2

img, target = test_set[1]
img = cv2.resize(img, (384, 384))
img = np.float32(img / 255)
img = np.moveaxis(img, [2], [0])
img = np.expand_dims(img, axis=0)

model_dir = Path('models')

model.to_onnx((model_dir / 'frcnn.onnx').as_posix(),
              input_sample=torch.zeros(1, 3, 384, 384),
              opset_version=11)

In [ ]:
import onnxruntime
import numpy as np
import cv2

session = onnxruntime.InferenceSession('models/frcnn.onnx')
input_name = session.get_inputs()[0].name
output_names = list(map(lambda x: x.name, session.get_outputs()))

In [ ]:
img, target = test_set[4]
img = cv2.resize(img, (384, 384))
img = np.float32(img / 255)
img = np.moveaxis(img, [2], [0])
img = np.expand_dims(img, axis=0)

result = session.run(output_names, {input_name: img})